# Extract notes from Zotero references

Python 3+, to install:

```
pip install git+https://github.com/sdaza/zotnote.git
```

In [1]:
import pandas as pd
import config
import zotnote as zn

In [8]:
obesity_big_paper = "NFWFUHKR"
epi_genetics = "M8N2VMAP"
testing = "VT3GKXCP"

In [9]:
zn.exportNotes(collection=testing, file="testing.csv",
    library_id=config.library_id, api_key=config.api_key) 

Notes saved in  testing.csv


In [10]:
pd.read_csv("testing.csv")

,id,citation,tags,title,reviewer,research_question
0,5NN8AW8B,Chen et al. 2018,NaN,Obesity trend in the United States and economi...,sebastian,testing


In [11]:
# zn.exportNotes(collection=obesity_big_paper, file="testing.csv",
#     library_id=config.library_id, api_key=config.api_key)
# pd.read_csv("testing.csv")

In [5]:
# testing
from pyzotero import zotero
zot = zotero.Zotero(config.library_id, config.library_type, config.api_key)
items = zot.collection_items("M8N2VMAP")

In [6]:
items = zot.item("U4MNK7QT")

In [4]:
# items